# Dataset processing

In [1]:
%load_ext watermark
%matplotlib inline

import glob

import cf_xarray as cfxr
import dcpy
import distributed
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Image

import eddydiff as ed
import xarray as xr

xr.set_options(keep_attrs=True)

plt.rcParams["figure.dpi"] = 140
plt.rcParams["savefig.dpi"] = 200
plt.style.use("ggplot")

%watermark -iv

matplotlib : 3.4.1
xarray     : 0.17.1.dev3+g48378c4b1
numpy      : 1.20.2
distributed: 2021.6.2
dcpy       : 0.1
cf_xarray  : 0.4.1.dev21+gab9dc66
eddydiff   : 0.1



## Groeskamp et al (2020)

In [2]:
from scipy.io import loadmat

In [11]:
import h5py

file = h5py.File("/home/deepak/datasets/groeskamp2020/WOA18_K.mat")

In [35]:
file0 = h5py.File("/home/deepak/datasets/groeskamp2020/WOA18_K0.mat")

In [37]:
file0["WOA18_K0"].keys()

<KeysViewHDF5 ['K0_WOA']>

In [44]:
ds = xr.Dataset(
    coords={
        "depth": (
            "depth",
            np.array(file["WOA18_K"]["z"]).squeeze(),
            {"standard_name": "depth", "units": "m", "positive": "down"},
        ),
        "lat": (
            "lat",
            np.array(file["WOA18_K"]["y"]).squeeze(),
            {"standard_name": "latitude", "units": "degrees_north"},
        ),
        "lon": (
            "lon",
            np.array(file["WOA18_K"]["x"]).squeeze(),
            {"standard_name": "longitude", "units": "degrees_east"},
        ),
    }
)

ds["Ke_0"] = (
    ("depth", "lat", "lon"),
    file0["WOA18_K0"]["K0_WOA"],
    {"description": "unsuppressed diffusivities", "long_name": "$K_e$"},
)
ds["Ke"] = (("depth", "lat", "lon"), file["WOA18_K"]["K_WOA"], {"long_name": "$K_e$"})
ds["S"] = (
    ("depth", "lat", "lon"),
    file["WOA18_K"]["S_WOA"],
    {"long_name": "$S$", "description": "suppression factor"},
)

In [45]:
ds.to_netcdf("/home/deepak/datasets/groeskamp2020/groeskamp2020.nc")